In [18]:
from binance.client import Client
import pandas as pd
import os
from dotenv import load_dotenv
from datetime import datetime, timedelta

In [19]:
# Load .env
load_dotenv()

# Get API key
api_key = os.getenv('BINANCE_API_KEY')
api_secret = os.getenv('BINANCE_API_SECRET')

In [26]:
# Create a Binance client
client = Client(api_key, api_secret)

def fetch_binance_data(symbol='BTCUSDT', interval='1d'):
    """
    Get history OHLCV data for Bitcoin (BTC)
    :param symbol:  'BTCUSDT'
    :param interval: K-line period, '1d'
    :return: pandas DataFrame
    """
    # Calculate the date from 5 years ago and format it in 'YYYY-MM-DD UTC' format
    five_years_ago = datetime.now() - timedelta(days=5*365)
    start_date = five_years_ago.strftime('%Y-%m-%d')
    
    # Get historical data
    klines = client.get_historical_klines(symbol, interval, start_str=start_date)

    # Convert data to -- pandas DataFrame
    ohlcv_data = pd.DataFrame(klines, columns=[
        'timestamp', 'open', 'high', 'low', 'close', 'volume', 
        'close_time', 'quote_asset_volume', 'number_of_trades', 
        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    ])

    # Converts the timestamp to date format
    ohlcv_data['timestamp'] = pd.to_datetime(ohlcv_data['timestamp'], unit='ms')

   # Add Unix timestamp (seconds since epoch) in scientific notation
    ohlcv_data['timestamp_unix'] = ohlcv_data['timestamp'].apply(lambda x: "{:.6e}".format(x.timestamp()))

    # return
    return ohlcv_data[['timestamp', 'timestamp_unix', 'open', 'high', 'low', 'close', 'volume']]


# Last three years of historical data
symbol = 'BTCUSDT' 
data = fetch_binance_data(symbol=symbol, interval='1d')

# output data -- 5
print(data.head())

# Save as CSV
output_filename = "bitcoin_5_years_data.csv"
data.to_csv(output_filename, index=False)

   timestamp timestamp_unix           open           high            low  \
0 2019-12-23   1.577059e+09  7500.71000000  7695.38000000  7265.84000000   
1 2019-12-24   1.577146e+09  7317.30000000  7436.68000000  7157.04000000   
2 2019-12-25   1.577232e+09  7255.77000000  7271.77000000  7128.86000000   
3 2019-12-26   1.577318e+09  7205.01000000  7435.00000000  7157.12000000   
4 2019-12-27   1.577405e+09  7202.00000000  7275.86000000  7076.42000000   

           close          volume  
0  7317.09000000  68051.99720300  
1  7255.77000000  43629.49418800  
2  7204.63000000  27492.04432300  
3  7202.00000000  36259.76107600  
4  7254.74000000  33642.70186100  
